In [1]:
import tensorflow as tf
import numpy as np
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
sys.path.append("../")

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from vae import VAE
from config import FLAGS
from batchloader import BatchLoader

In [3]:
def train():
    save_dir = "model"
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    tf.reset_default_graph()
    
    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
        batchloader = BatchLoader()
        with tf.variable_scope("VAE"):
            vae = VAE(sess=sess, batchloader=batchloader, learning_rate=FLAGS.LEARNING_RATE, training=True, ru=False)
        
        with tf.variable_scope("VAE", reuse=True):
            vae_test = VAE(sess=sess, batchloader=batchloader, learning_rate=FLAGS.LEARNING_RATE, training=False, ru=True)
        
        saver = tf.train.Saver()
        summary_writer = tf.summary.FileWriter(FLAGS.LOG_DIR, sess.graph)
        
        sess.run(tf.global_variables_initializer())
        
        loss_sum = []
        reconst_loss_sum = []
        kld_sum = []
        
        step = 0
        lr = FLAGS.LEARNING_RATE
        
        train_batch = batchloader.make_batch(FLAGS.BATCH_SIZE, is_training=True)
        total_batch = int(len(train_batch) / FLAGS.BATCH_SIZE)
        
        val_batch_idx = 0
        best_loss = 10000
        patient = 0

        print("Start Learning!!")
        for epoch in range(FLAGS.EPOCH):
                
            for batch in range(total_batch):
                step += 1
                
                kld_weight = (math.tanh((step-3500)/1000) + 1) / 2
                
                batch_idx = batch*FLAGS.BATCH_SIZE
                minibatch = batch[batch_idx:batch_idx+FLAGS.BATCH_SIZE]
                encoder_input, decoder_input, target = batchloader.prepro_minibatch(minibatch, dropword=True)
                #encoder_input, decoder_input, target = batchloader.next_batch(FLAGS.BATCH_SIZE, batch_idx=batch, is_training=True)
                
                feed_dict = {vae.encoder_input: encoder_input,
                             vae.decoder_input: decoder_input,
                             vae.target: target,
                             vae.kld_weight: kld_weight,
                             vae.step: step,
                             vae.learning_rate: lr}
                
                logits, loss, reconst_loss, kld, merged_summary, _ = sess.run([vae.logits, vae.loss, vae.reconst_loss, vae.kld, vae.merged_summary, vae.train_op], 
                                                                              feed_dict=feed_dict)
                
                reconst_loss_sum.append(reconst_loss)
                kld_sum.append(kld)
                loss_sum.append(loss)
                summary_writer.add_summary(merged_summary, step)
                
                if batch%100 == 99:
                    avg_loss = np.average(loss_sum)
                    avg_reconst_loss = np.average(reconst_loss_sum)
                    avg_kld = np.average(kld_sum)

                    print('[Epoch {}] loss: {}, reconst_loss: {}, kld: {}'.format(epoch, avg_loss, avg_reconst_loss, avg_kld))
                    loss_sum = []
                    reconst_loss_sum = []
                    kld_sum = []

                    sample_train_input = sess.run([vae.encoder_input], feed_dict=feed_dict)
                    encoder_input_texts = batchloader.logits2str(sample_train_input, 1, onehot=False, numpy=True)

                    sample_train_outputs = batchloader.logits2str(logits, 1)


                    print('train input: {}, train output: {}'.format(encoder_input_texts[0], sample_train_outputs[0]))
                
                
            
            
            #validation output
            val_batch = batchloader.make_batch(FLAGS.BATCH_SIZE, is_training=False)
            val_minibatch = val_batch[val_batch_idx*FLAGS.BATCH_SIZE:(val_batch_idx + 1)*FLAGS.BATCH_SIZE]
            val_encoder_input, val_decoder_input, val_target = batchloader.prepro_minibatch(val_minibatch, dropword=False)
            #sample_input, _, sample_target = batchloader.next_batch(FLAGS.BATCH_SIZE, is_training=False)
            val_encoder_input_list, val_latent_variables = sess.run([vae_test.encoder_input_list, vae_test.encoder.latent_variables],
                                                                  feed_dict={vae_test.encoder_input: val_encoder_input})

            val_logits, val_loss, val_merged_summary = sess.run([vae_test.logits, vae_test.reconst_loss, vae_test.merged_summary],
                                                 feed_dict = {vae_test.encoder_input: val_encoder_input,
                                                              vae_test.decoder_input: val_decoder_input,
                                                              vae_test.target: val_target,
                                                              vae_test.latent_variables: val_latent_variables,
                                                              vae_test.kld_weight: kld_weight})

            if val_loss < best_loss:
                best_loss = val_loss
            else:
                if patient == 10:
                    exit = 1
                patient += 1

            print('[Epoch {}] best loss: {}, current loss: {}, patient: {}'.format(epoch, best_loss, val_loss, patient))

            val_input_texts = batchloader.logits2str(val_encoder_input_list, 1, onehot=False, numpy=True)
            val_output_texts = batchloader.logits2str(val_logits, 1)

            print('sample input: {}, sample output: {}'.format(val_input_texts[0], val_output_texts[0]))
            summary_writer.add_summary(merged_summary, step)
            
            if exit == 1:
                filename = os.path.join(save_dir, "model_{}.ckpt".format(epoch))
                #save model
                save_path = saver.save(sess, filename)
                print('Model saved in file {}'.format(save_path))
                print("Finish Learning!!")
                break

            val_batch_idx += 1

In [4]:
if __name__ == "__main__":
    sys.argv = ['ddd']
    print(FLAGS.LEARNING_RATE)
    train()

0.001
sss
(32, 60, 353)
Instructions for updating:
Use the retry module or similar alternatives.


TypeError: Failed to convert object of type <class 'tensorflow.python.ops.rnn_cell_impl.DropoutWrapper'> to Tensor. Contents: <tensorflow.python.ops.rnn_cell_impl.DropoutWrapper object at 0x7f777cc2b9b0>. Consider casting elements to a supported type.